# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   879.592450  0.770986  1234  880.363436  878.821464  1760.726871   
6   605.718452  0.630315  1234  606.348767  605.088136  1212.697534   
7   235.957822 -0.231754  1234  235.726068  236.189576   471.452136   
8   363.190340 -0.572203  1234  362.618138  363.762543   725.236275   
9   956.027244 -0.699718  1234  955.327526  956.726962  1910.655053   
10   40.914043 -0.812279  1234   40.101764   41.726322    80.203528   
11  745.392825 -0.187544  1234  745.205281  745.580369  1490.410562   
12  583.434617  0.227586  1234  583.662203  583.207031  1167.324406   
13  868.083190  0.999492  1234  869.082682  867.083697  1738.165364   
14  187.878777 -0.624611  1234  187.254167  188.503388   374.508333   

              f  
5   1757.642927  
6   1210.176273  
7    472.379153  
8    727.525086  
9   1913.453925  
10    83.452643  
11  1491.160738  
12  1166.414062  
13  1734.167394  
14   377.006776

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -880.363436   878.821464
6   -606.348767   605.088136
7   -235.726068   236.189576
8   -362.618138   363.762543
9   -955.327526   956.726962
10   -40.101764    41.726322
11  -745.205281   745.580369
12  -583.662203   583.207031
13  -869.082682   867.083697
14  -187.254167   188.503388

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -880.363436   878.821464
6   -606.348767   605.088136
7   -235.726068   236.189576
8   -362.618138   363.762543
9   -955.327526   956.726962
10   -40.101764    41.726322
11  -745.205281   745.580369
12  -583.662203   583.207031
13  -869.082682   867.083697
14  -187.254167   188.503388

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-880.36343555,  878.82146369],
       [-606.34876719,  605.08813641],
       [-235.72606813,  236.18957632],
       [-362.6181376 ,  363.76254301],
       [-955.3275264 ,  956.72696229],
       [ -40.10176404,   41.72632153],
       [-745.20528123,  745.58036883],
       [-583.66220314,  583.20703094],
       [-869.08268189,  867.08369712],
       [-187.2541667 ,  188.50338801]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1758.726871  -1757.642927
6    1210.697534  -1210.176273
7     469.452136   -472.379153
8     723.236275   -727.525086
9    1908.655053  -1913.453925
10     78.203528    -83.452643
11   1488.410562  -1491.160738
12   1165.324406  -1166.414062
13   1736.165364  -1734.167394
14    372.508333   -377.006776

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf